In [4]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
from scipy import constants as C
import IPython.display

base_diagnostic = 'out/test'
searay_object_name = 'glass'
#searay_wave_type = 'paraxial'
searay_wave_type = 'uppe'

mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 18
l1_mm = 1e3*0.8e-6/(2*np.pi)
t1_ps = 1e12*0.8e-6/(2*np.pi)/C.c

real_field = searay_wave_type=='uppe'

A = np.load(base_diagnostic+'_'+searay_object_name+'_'+searay_wave_type+'_wave.npy')
data_ext = np.load(base_diagnostic+'_'+searay_object_name+'_'+searay_wave_type+'_plot_ext.npy')
data_ext = np.concatenate((data_ext[0:2],l1_mm*data_ext[2:4]))
print('Shape of field array =',A.shape)
print('Bounding box =',data_ext)

frame_w = ipywidgets.IntSlider(min=0,max=A.shape[3]-1,step=1,value=0,continuous_update=False)
disp_w = ipywidgets.Dropdown(options=['Falsecolor2D','Lineout'],value='Falsecolor2D',description='Display')
rep_w = ipywidgets.Dropdown(options=['Time','Fourier'],value='Time',description='Representation')

def get_freq_time_data(A,ext):
    if real_field:
        # Nodes are like [0,1,2,3] and walls are like [-0.5,0.5,1.5,2.5,3.5]
        # In these examples the user's requested upper bound would be 4 (it is thrown out)
        # Therefore element N/2+1 should be regarded as the central frequency
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        return dw,wc,tmax
    else:
        # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        return dw,wc,tmax

def change_rep_data(rep):
    global plot_data,hstr,vstr,cstr,plot_ext
    plot_ext = np.copy(data_ext)
    if rep=='Time' and real_field:
        dw,wc,tmax = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = t1_ps*tmax
        hstr = r'$t-z/c$ (ps)'
        vstr = r'$x$ (mm)'
        cstr = r'$A(t)$'
        plot_data = np.fft.irfft(np.conj(A),axis=0)
        # Shift the time reference to box center
        # Note that time is referenced to the largest amplitude ray at the input surface
        plot_data = np.roll(plot_data,int(plot_data.shape[0]/2),axis=0)
    if rep=='Time' and not real_field:
        dw,wc,tmax = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = t1_ps*tmax
        hstr = r'$t-z/c$ (ps)'
        vstr = r'$x$ (mm)'
        cstr = r'$|a|(t)$'
        plot_data = np.abs(np.fft.ifft(np.fft.ifftshift(np.conj(A),axes=0),axis=0))
        # Shift the time reference to box center
        # Note that time is referenced to the largest amplitude ray at the input surface
        plot_data = np.roll(plot_data,int(plot_data.shape[0]/2),axis=0)
    if rep=='Fourier':
        hstr = r'$\omega/\omega_0$'
        vstr = r'$x$ (mm)'
        cstr = r'$|a|(\omega)$'
        plot_data = np.abs(A)
        #plot_data = np.log10(plot_data+1e-20)

def change_rep(rep):
    change_rep_data(rep)
    disp_frame(frame_w.value,disp_w.value,rep)

def disp_frame(frame,disp,rep):
    tslice = np.int(A.shape[0]/2)
    yslice = np.int(A.shape[2]/2)
    plt.figure(1,figsize=(16,7))
    if disp=='Lineout':
        x = np.linspace(plot_ext[0],plot_ext[1],plot_data.shape[0])
        plt.plot(x,plot_data[:,yslice,yslice,frame])
        #x = np.linspace(plot_ext[2],plot_ext[3],plot_data.shape[1])
        #plt.plot(x,plot_data[tslice+4,:,yslice,frame])
    else:
        plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),vmin=-.001,vmax=.001,origin='lower',cmap='viridis',aspect=1.0,extent=plot_ext)
        #plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),vmin=np.min(plot_data),vmax=np.max(plot_data),origin='lower',cmap='viridis',aspect='auto',extent=plot_ext)
        b=plt.colorbar()
        b.set_label(cstr,size=24)
    plt.xlabel(hstr,size=24)
    plt.ylabel(vstr,size=24)
    plt.tight_layout()
    plt.show()

rep_iw = ipywidgets.interactive(change_rep,rep=rep_w)
disp_iw = ipywidgets.interactive(disp_frame,frame=frame_w,disp=disp_w,rep=rep_w)
change_rep_data('Time')
IPython.display.display(disp_iw)

Shape of field array = (1024, 64, 1, 9)
Bounding box = [-3.90625000e-03  7.99609375e+00  0.00000000e+00  3.00000000e-01]


interactive(children=(IntSlider(value=0, continuous_update=False, description='frame', max=8), Dropdown(descri…